In [2]:
import os
import sys
import re
import shutil
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from shutil import copyfile

## Yue

In [50]:
!./download_script.sh KP 223797 223843

Hello! I am downloading data. Please wait.
^C


In [52]:
%ls

download_script.sh*  file_divide.ipynb  Yue/


In [53]:
r_880, r_463 = os.listdir('Yue')

### Dealing with R880F

In [73]:
le_files = sorted(os.listdir(f'Yue/{r_880}/LEseq/data'))
le_files[0:5]

['KP223797.gb', 'KP223798.gb', 'KP223799.gb', 'KP223800.gb', 'KP223801.gb']

In [74]:
def convert_gb_fa(gb_handle, fa_handle):
    for seq_record in SeqIO.parse(gb_handle, "genbank") :
        print("Dealing with GenBank record %s" % seq_record.id)
        for seq_feature in seq_record.features:
            if seq_feature.type=="CDS" :
                assert len(seq_feature.qualifiers['translation'])==1
                header = ">db|{0}|{1}_NCOV {2} OS={3}\n"
                dscrptn = [seq_record.name,
                           seq_feature.qualifiers['protein_id'][0],
                           seq_feature.qualifiers['product'][0],
                           seq_record.annotations['organism']]
                fa_handle.write(header.format(*dscrptn))
                fa_handle.write("{}\n".format(seq_feature.qualifiers['translation'][0]))

In [75]:
def date_convert(date):
    # month_dict = {}
    if len(date) == 11:
        day = date[0:2]
        month = date[3:6]
        year = date[7:]
        pass
    else:
        print('Oops')
        return

In [76]:
div_dict = {}

for file in le_files:
    for seq_record in SeqIO.parse(f'Yue/{r_880}/LEseq/data/{file}',"genbank"):
        # print(seq_record.id)
        # for seq_feature in seq_record.features:
            # if seq_feature.type == 'source':
                # print(seq_feature.qualifiers['collection_date'])
        seq_id = seq_record.id[:len(seq_record.id)-1] + 'gb'
        for seq_feature in seq_record.features:
            if seq_feature.type == 'source':
                date = seq_feature.qualifiers['collection_date'][0]
                #print(date)
                if date in div_dict:
                    div_dict[date].append(seq_id)
                else:
                    div_dict[date] = [seq_id]
                    
#div_dict

In [77]:
for date in div_dict.keys():
    if not os.path.isdir(f'Yue/{r_880}/LEseq/{date}'):
        os.mkdir(f'Yue/{r_880}/LEseq/{date}')
    for file in div_dict[date]:
        copyfile(f'Yue/{r_880}/LEseq/data/{file}', f'Yue/{r_880}/LEseq/{date}/{file}')

### Dealing with R463F

In [78]:
le_files = sorted(os.listdir(f'Yue/{r_463}/LEseq/data'))
le_files[0:5]

['KP223729.gb', 'KP223730.gb', 'KP223731.gb', 'KP223732.gb', 'KP223733.gb']

In [79]:
div_dict = {}

for file in le_files:
    for seq_record in SeqIO.parse(f'Yue/{r_463}/LEseq/data/{file}',"genbank"):
        # print(seq_record.id)
        # for seq_feature in seq_record.features:
            # if seq_feature.type == 'source':
                # print(seq_feature.qualifiers['collection_date'])
        seq_id = seq_record.id[:len(seq_record.id)-1] + 'gb'
        for seq_feature in seq_record.features:
            if seq_feature.type == 'source':
                date = seq_feature.qualifiers['collection_date'][0]
                #print(date)
                if date in div_dict:
                    div_dict[date].append(seq_id)
                else:
                    div_dict[date] = [seq_id]
                    
#div_dict

In [80]:
for date in div_dict.keys():
    if not os.path.isdir(f'Yue/{r_463}/LEseq/{date}'):
        os.mkdir(f'Yue/{r_463}/LEseq/{date}')
    for file in div_dict[date]:
        copyfile(f'Yue/{r_463}/LEseq/data/{file}', f'Yue/{r_463}/LEseq/{date}/{file}')

## Turnbull

In [3]:
le_files = sorted(os.listdir(f'Turnbull'))
le_files[0:5]

['HM586107.gb', 'HM586108.gb', 'HM586109.gb', 'HM586110.gb', 'HM586111.gb']

In [5]:
date_day_dict = {}

i = 0

for file in le_files:
    for seq_record in SeqIO.parse(f'Turnbull/{file}',"genbank"):
        for seq_feature in seq_record.features:
            
            if seq_feature.type == 'source':
                
                #print(seq_feature.qualifiers['isolate'][0])
                
                patient = re.search(r'MM\d\d', seq_feature.qualifiers['isolate'][0])[0]
                date_day_dict[patient] = {}
                
                if re.search(r'd[\d]*', seq_feature.qualifiers['isolate'][0]) is not None:
                    day = re.search(r'd[\d]*', seq_feature.qualifiers['isolate'][0])[0]
                else:
                    if re.search(r'D[\d]*', seq_feature.qualifiers['isolate'][0]) is not None:
                        day = re.search(r'D[\d]*', seq_feature.qualifiers['isolate'][0])[0].lower()
                
                if not os.path.isdir(f'Turnbull/{patient}'):
                    os.mkdir(f'Turnbull/{patient}')
                
                try:
                    date = seq_feature.qualifiers['collection_date'][0]
                    if day not in date_day_dict:
                        date_day_dict[patient][day] = date
                except KeyError:
                    if day in date_day_dict:
                        date = date_day_dict[patient][day]
                
                if not os.path.isdir(f'Turnbull/{patient}/{date}'):
                    os.mkdir(f'Turnbull/{patient}/{date}')
                
                shutil.move(f'Turnbull/{file}', f'Turnbull/{patient}/{date}')

#### Division for proteins

In [70]:
date_day_dict = {}

i = 0

for file in le_files:
    for seq_record in SeqIO.parse(f'Turnbull/all_gb_files/{file}',"genbank"):
        for seq_feature in seq_record.features:
            #print(seq_feature)
            if seq_feature.type == 'source':
                
                #print(seq_feature.qualifiers['isolate'][0])
                
                patient = re.search(r'MM\d\d', seq_feature.qualifiers['isolate'][0])[0]
                date_day_dict[patient] = {}
                
                if re.search(r'd[\d]*', seq_feature.qualifiers['isolate'][0]) is not None:
                    day = re.search(r'd[\d]*', seq_feature.qualifiers['isolate'][0])[0]
                else:
                    if re.search(r'D[\d]*', seq_feature.qualifiers['isolate'][0]) is not None:
                        day = re.search(r'D[\d]*', seq_feature.qualifiers['isolate'][0])[0].lower()
                
                if not os.path.isdir(f'Turnbull/{patient}'):
                    os.mkdir(f'Turnbull/{patient}')
                
                try:
                    date = seq_feature.qualifiers['collection_date'][0]
                    if day not in date_day_dict:
                        date_day_dict[patient][day] = date
                except KeyError:
                    if day in date_day_dict:
                        date = date_day_dict[patient][day]
                
                if not os.path.isdir(f'Turnbull/{patient}/{date}'):
                    os.mkdir(f'Turnbull/{patient}/{date}')
            
            if seq_feature.type == 'CDS':
                
                gene_name = seq_feature.qualifiers['gene'][0]
                
                if not os.path.isdir(f'Turnbull/{patient}/{date}/{gene_name}'):
                    os.mkdir(f'Turnbull/{patient}/{date}/{gene_name}')
                    
                try:
                    protein_id = seq_feature.qualifiers['protein_id'][0]
                except KeyError:
                    protein_id = 'no_protein_ID'
                    
                try:
                    protein = seq_feature.qualifiers['translation'][0]
                    with open(f'Turnbull/{patient}/{date}/{gene_name}/{file}.fasta', 'w') as fasta_file:
                        SeqIO.write(SeqRecord(Seq(protein), id=f'{gene_name}_{day}', description=protein_id), fasta_file, 'fasta')
                except KeyError:
                    continue

In [79]:
for file in le_files:
    for seq_record in SeqIO.parse(f'Turnbull/all_gb_files/{file}',"genbank"):
        for seq_feature in seq_record.features:
            #print(seq_feature)
            #print(str(seq_feature.location))
            if seq_feature.type == 'source':
                if int(re.search(r':[\d]*', str(seq_feature.location))[0][1:]) > 8300:
                    
                    patient = re.search(r'MM\d\d', seq_feature.qualifiers['isolate'][0])[0]
                    
                    if not os.path.isdir(f'Turnbull/{patient}/genomewide'):
                        os.mkdir(f'Turnbull/{patient}/genomewide')
                    
                    shutil.copyfile(f'Turnbull/all_gb_files/{file}', f'Turnbull/{patient}/genomewide/{file}')
                    

In [80]:
for date in div_dict.keys():
    if not os.path.isdir(f'Yue/{r_463}/LEseq/{date}'):
        os.mkdir(f'Yue/{r_463}/LEseq/{date}')
    for file in div_dict[date]:
        copyfile(f'Yue/{r_463}/LEseq/data/{file}', f'Yue/{r_463}/LEseq/{date}/{file}')